In [1]:
%matplotlib inline

from keras.models import load_model
from keras.models import model_from_json
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 

Using TensorFlow backend.


In [2]:
model_location = '/home/sage/ml_playground/catsvsdogs/trained/keras/vgg16_cat_dog_retrained.json'
weights_location = '/home/sage/ml_playground/catsvsdogs/trained/keras/vgg16_cat_dog_retrained.h5'

with open(model_location, 'r') as myfile:
    model_json = myfile.read()

model = model_from_json(model_json)
model.load_weights(weights_location)

headless_model = Model(inputs=model.input, outputs=model.get_layer('fc1').output)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [3]:
data_dir = '/home/sage/ml_playground/catsvsdogs/PetImages/tsne'
img_width, img_height = 224, 224

imgs_generator = ImageDataGenerator().flow_from_directory(directory=data_dir,
                                                              target_size=[img_width, img_height],
                                                              batch_size=1,
                                                              shuffle=False,
                                                              class_mode='categorical')

Found 5000 images belonging to 2 classes.


In [4]:
from tqdm import tqdm

n_imgs = 5000

filenames = []
vectors = []
samples = []

for i in tqdm(range(n_imgs)):
    sample, _ = next(imgs_generator)
    filename = imgs_generator.filenames[imgs_generator.batch_index - 1]
    filenames.append(filename)
    vectors.append(headless_model.predict(sample)[0])

100%|██████████| 5000/5000 [10:58<00:00,  7.59it/s]


In [6]:
filenames_and_vectors = {}
filenames_and_vectors['files'] = np.asarray(filenames)
filenames_and_vectors['data'] = np.asarray(vectors)

In [7]:
from pickle import dump

with open('/home/sage/ml_playground/catsvsdogs/trained/keras/filenames_and_vectors.pkl', 'wb') as f:
    dump(filenames_and_vectors, f)